In [1]:
from scipy.optimize import minimize
from scipy.optimize import curve_fit
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.path as mpltPath
import math as math

In [93]:
def powerlaw(r, s0, b):  
    # r0 no es variable aleatoria
    r0 = 300
    return s0*(r/r0)**(-b)

def baricentro(signal,det_pos):
    bari_x = signal[0]*det_pos[0][0]
    bari_y = signal[0]*det_pos[0][1]
    sum_s = np.sum(signal)
    for i in range(1,len(signal)):
        bari_x += signal[i]*det_pos[i][0]
        bari_y += signal[i]*det_pos[i][1]
    return(bari_x/sum_s,bari_y/sum_s)
    


#estimacion inicial del core es el baricentro

def funcion_costo2(theta,x_pos,y_pos,signal_sim,zenith,azimuth,ysigma):

        s0 = theta[0]
        b = theta[1]
        x_core = theta[2]
        y_core = theta[3]
        
        costo = np.zeros_like(s0)

        for (dist_x,dist_y,y1,sigma1) in zip(x_pos,y_pos,signal_sim,ysigma):
            dist = getDistances(dist_x,dist_y,x_core,y_core,zenith,azimuth)[0]
            yfit = powerlaw(dist, s0, b)
            residuo = y1 - yfit
            z = (residuo / sigma1)**2
            costo += z

        return costo
    
costo2 = lambda theta: funcion_costo2(theta,x_det,y_det,y_random,zenith,azimuth,sigma) 

def in_hex(poli):
    path = mpltPath.Path(poli)
    hex = False
    random_point = (np.random.uniform(-433,433),np.random.uniform(-375,375))
    while hex != True:
        if path.contains_points([random_point]) == True:
            hex = True
        else:
            random_point = (np.random.uniform(-433,433),np.random.uniform(-375,375))
    return random_point

def getDistances(xpositions, ypositions, x0=0, y0=0, zenithRad=0, azimuthRad=0):

    dx = xpositions - x0    # detector wrt core positions
    dy = ypositions - y0
    
    groundDistances = np.sqrt(dx*dx+dy*dy)

    # Direction cosines in the plane xy
    dirx =  math.cos(azimuthRad)
    diry =  math.sin(azimuthRad)

    axisProjections = math.sin(zenithRad) * (dx*dirx+dy*diry)
    
    distances = np.sqrt( groundDistances**2 - axisProjections**2 )
    
    return (distances,axisProjections,groundDistances)

def simZenith(zenithMinDeg, zenithMaxDeg) :

    zenithMinRad = math.radians(zenithMinDeg)
    sin2MinZenith = math.sin(zenithMinRad)**2

    zenithMaxRad = math.radians(zenithMaxDeg)
    sin2MaxZenith = math.sin(zenithMaxRad)**2

    rng = np.random.default_rng()
    u = rng.random() 
    
    sin2Zenith = sin2MinZenith + u * (sin2MaxZenith-sin2MinZenith)
    sinZenith = math.sqrt(sin2Zenith) 

    zenith = math.asin(sinZenith)
    azimut = np.random.uniform(0,2*np.pi)
    
    return (zenith,azimut)

poligono = [[-216.5,-375],[216.5,-375],[433,0],[216.5,375],[-216.5,375],[-433,0]]
detectores_pos = [[-216.5,-375],[216.5,-375],[433,0],[216.5,375],[-216.5,375],[-433,0],[0,0]]
rectangulo = [[-433,-375],[433,-433],[433,375],[-433,375]]
pointsx,pointsy = [-216.5,216.5,433,216.5,-216.5,-433.5,0] , [-375,-375,0,375,375,0,0]
random_point = in_hex(poligono)
centro = (0,0)
x_det = np.zeros(len(detectores_pos))
y_det = np.zeros(len(detectores_pos))
for i in range(len(detectores_pos)):
    x_det[i] = detectores_pos[i][0]
    y_det[i] = detectores_pos[i][1]

In [102]:
#Defino parametros de LDF 
r0 = 300 #metros 
s0 = 35 #VEM
angulo = 20
zenith = simZenith(-angulo,angulo)[0]
azimuth = simZenith(-angulo,angulo)[1]
b = 2.1    

#Genero el punto random
random_point = in_hex(poligono)

#calculo las distancias entre el evento y los 7 detectores.
distancias = getDistances(np.asarray(pointsx),np.asarray(pointsy),random_point[0],random_point[1],simZenith(-angulo,angulo)[0],simZenith(-angulo,angulo)[1])
r = np.asarray(distancias[0])

signal = powerlaw(r,s0,b)
sigma_signal = np.sqrt(signal)

#Hago el ajuste y defino la semilla inicial
y_random = np.random.normal(signal,sigma_signal,size = len(signal)) #signal
y_random = y_random[y_random > 2]
sigma = np.sqrt(y_random)

core_est_x = baricentro(y_random,detectores_pos)[0] 
core_est_y = baricentro(y_random,detectores_pos)[1] 

In [103]:
res = minimize(costo2, x0=(30,1,core_est_x,core_est_y),tol=1e-10)
print('Minimize results')
print("Parametros ajustados: ", res.x)
cova = 2*res.hess_inv
par_error = np.sqrt(np.diagonal(cova))
print("Errores de los parámetros", par_error)
print(cova)

Minimize results
Parametros ajustados:  [  28.74938678    1.39583382 -177.38338717 -330.0434171 ]
Errores de los parámetros [ 7.85326854  0.88257812  0.24190009 92.7711594 ]
[[ 6.16738267e+01  6.50348367e+00 -1.62290133e+00  6.95801559e+02]
 [ 6.50348367e+00  7.78944143e-01 -1.94406444e-01  8.17061764e+01]
 [-1.62290133e+00 -1.94406444e-01  5.85156517e-02 -2.03991684e+01]
 [ 6.95801559e+02  8.17061764e+01 -2.03991684e+01  8.60648802e+03]]


In [104]:
random_point

(-159.19153189051514, -269.7850213794043)

In [85]:
np.zeros_like(1)

array(0)